## Note
Before starting the jupyter server, add to spark(static) the [lab1_lib](http://kompics.sics.se/maven/snapshotrepository/se/kth/spark/lab1_lib_2.11/1.0-SNAPSHOT/lab1_lib_2.11-1.0-SNAPSHOT.jar) library.

In [ ]:
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.UnaryTransformer
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.linalg.Vectors
import scala.collection.mutable.WrappedArray
import org.apache.spark.hack.VectorType
import se.kth.spark.lab1.Array2Vector
import se.kth.spark.lab1.Obs

## Note
Replace the filePath to point to your project/dataset

In [ ]:
val filePath = "hdfs:///Projects/id2223_lab1/million/millionsong.txt"
val rawDF = sc.textFile(filePath).toDF("raw").cache()
rawDF.show(5)

In [ ]:
val obsValues = sc.textFile(filePath).map(s=>s.split(",")).map(
    s=>Obs(s(0).replaceAll("\"", "").toDouble, 
            s(1).replaceAll("\"", "").toDouble,
            s(2).replaceAll("\"", "").toDouble,
            s(3).replaceAll("\"", "").toDouble
        )
)
obsValues.toDF().registerTempTable("songs")

In [ ]:
%%sql 
select * from songs limit 10

In [ ]:
val regexTokenizer = new RegexTokenizer().setInputCol("raw").setOutputCol("arraytokens").setPattern(",")
val tokenizerDF = regexTokenizer.transform(rawDF)
tokenizerDF.show(5)
    
val arr2Vect = new Array2Vector().setInputCol(regexTokenizer.getOutputCol).setOutputCol("vectortokens") 
val array2VectorDF = arr2Vect.transform(tokenizerDF)
array2VectorDF.show(5)

## Note
If you want to declare a new class, you need to declare it inside an object, otherwise some imports might not work.

In [ ]:
object UnaryTransformerExample {
    class Array2Vector(override val uid: String)extends UnaryTransformer[WrappedArray[String], Vector, Array2Vector] {

        def this() = this(Identifiable.randomUID("arrayToVector"))

        override protected def createTransformFunc: WrappedArray[String] => Vector = {
            (p1: WrappedArray[String]) => {Vectors.dense(p1.array.map { x => x.toDouble })}
        }

        override protected def outputDataType: VectorType = {
            new VectorType
        }
    }
}

In [ ]:
val arr2Vect3 = new UnaryTransformerExample.Array2Vector().setInputCol(regexTokenizer.getOutputCol).setOutputCol("vectortokens") 
val array2VectorDF3 = arr2Vect3.transform(tokenizerDF)
array2VectorDF3.show(5)